In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cf
import numpy as np

In [ ]:
# import fields and geostrophic wind

%store -r fields
[uWind,vWind,wWind,temp,geop,div,vort,geop_height] = fields

%store -r grid
[lon,lat,pressure_levels] = grid

%store -r uWindG
%store -r vWindG

The thermal wind is calculated according to
\begin{align}
\vec{v}_t(p_i) = \vec{v}_g(p_i) - \vec{v}_g(800hPa)
\end{align}

You can adjust the cell below (optional). The pressure index of the reference geostrophic wind in the calculation is 5 because pressure_levels[5] = 800hPa. Therefore all thermal wind velocities are calculated in reference to the 800hPa level. You can change that and calculate the thermal wind in reference to another pressure level.

Pay attention to this if you choose to download and use your own datasets.

In [ ]:
uWindT = np.zeros_like(uWindG)
vWindT = np.zeros_like(uWindT)
for p in range(1,len(pressure_levels)):
    uWindT[p,:,:] = uWindG[p,:,:] - uWindG[5,:,:]
    vWindT[p,:,:] = vWindG[p,:,:] - vWindG[5,:,:]

In [ ]:
def plot_wind(field,N=90,S=90,W=0,E=360,pressure_level=0,spacing=5,vmin=None,vmax=None):
    '''this function plots field with an areal extend of [N,S,W,E] at pressure_level
       with the wind field displayed as arrows.
       W is given in degrees east and has to be smaller than E,
       also no negative values are allowed.
       spacing is the space inbetween arrows in degrees.
       the geopotential height is displayed as contour lines.
       vmin and vmax are mainly thought to be used for divergence and vorticity
       of the wind field and can be ignored for other fields.
       the geopotential height is displayed as contour lines (in m).
    '''
    N = 90-N
    S = 90+S
    
    fig, ax = plt.subplots(figsize=(15,8), subplot_kw={'projection': ccrs.PlateCarree()})
    im = ax.contourf(lon[W:E], lat[N:S], field[1,pressure_level,N:S,W:E],
                    cmap='viridis', levels=50, vmin=vmin, vmax=vmax)
    
    im2 = ax.contour(lon[W:E], lat[N:S], geop_height[1,pressure_level,N:S,W:E])
    ax.clabel(im2, im2.levels, inline=True,colors='k')
    
    Q = ax.quiver(lon[W:E][::spacing], lat[N:S][::spacing],
                uWind[1,pressure_level,N:S,W:E][::spacing,::spacing],
                vWind[1,pressure_level,N:S,W:E][::spacing,::spacing])
    Qk = ax.quiverkey(Q,0.5,-0.1,np.nanmax(uWind[1,pressure_level,N:S,W:E][::spacing,::spacing]),
            label="{:.0f}".format(np.array(np.nanmax(uWind[1,pressure_level,N:S,W:E][::spacing,::spacing]))) + " m/s wind velocity",
            labelpos = 'E')
    ax.add_feature(cf.COASTLINE)
    ax.add_feature(cf.BORDERS)
    ax.set_xticks([0],[0])
    ax.set_yticks([0],[0])

    fig.colorbar(im, orientation='horizontal', fraction=0.039*len(lon)/len(lat),label=f"{field.long_name} [{field.units}]")
    ax.set_title(f"{field.long_name} at pressure level {pressure_levels[pressure_level]} hPa", fontsize=15)
    fig.tight_layout()

In [ ]:
def therm_wind(field,N=90,S=90,W=0,E=360,pressure_level=-2,spacing=5):
    '''this function plots field with an areal extend of [N,S,W,E] at pressure_level
       with the thermal wind displayed as arrows.
       W is given in degrees east and has to be smaller than E,
       also no negative values are allowed.
       spacing is the space inbetween arrows in degrees.
       the geopotential height is displayed as contour lines (in m).
    '''
    
    N = 90-N
    S = 90+S

    fig, ax = plt.subplots(figsize=(15,8), subplot_kw={'projection': ccrs.PlateCarree()})
    im = ax.contourf(lon[W:E], lat[N:S], field[1,pressure_level,N:S,W:E],
                        cmap='viridis', levels=20)
    
    im2 = ax.contour(lon[W:E], lat[N:S], geop_height[1,pressure_level,N:S,W:E])
    ax.clabel(im2, im2.levels, inline=True,colors='k')

    Q2 = ax.quiver(lon[W:E][::spacing], lat[N:S][::spacing],
                  uWindT[pressure_level,N:S,W:E][::spacing,::spacing],
                  vWindT[pressure_level,N:S,W:E][::spacing,::spacing])
    Qk2 = ax.quiverkey(Q2, 0.5,-0.15,np.nanmax(uWindT[pressure_level,N:S,W:E][::spacing,::spacing]),
                       label="{:.0f}".format(np.nanmax(uWindT[pressure_level,N:S,W:E][::spacing,::spacing])) + 'm/s thermal wind velocity',
                       labelpos = 'E')

    ax.add_feature(cf.COASTLINE)
    ax.add_feature(cf.BORDERS)
    ax.set_xticks([0],[0])
    ax.set_yticks([0],[0])

    fig.colorbar(im, orientation='horizontal', fraction=0.039*len(lon)/len(lat), label=f"{field.long_name} [{field.units}]")
    ax.set_title(f"{field.long_name} at p = {pressure_levels[pressure_level]} hPa", fontsize=20)
    fig.tight_layout()

### Use the function term_wind to display the thermal wind in different regions and at different altitudes

You can also use the function plot_wind to just display the wind vectors for comparison

The thermal wind velocities are all calculated relative to the 800hPa level

In [ ]:
pressure_levels # in hPa

In [ ]:
plot_wind(temp,N=10,S=80,W=280,E=360,pressure_level=5,spacing=4)

In [ ]:
plot_wind(temp,N=10,S=80,W=280,E=360,pressure_level=4,spacing=4)

In [ ]:
therm_wind(temp,N=10,S=80,W=280,E=360,pressure_level=4,spacing=4)